# Materials
* [Bunch of articles](http://www.mitpressjournals.org/doi/pdf/10.1162/neco.1993.5.6.954) - I strongly recomment this resource, cause it hosts most actual (by year of publishing) articles.
* [realization on C++](https://github.com/BelBES/ESOINN)
* [ESOINN algorithm](http://cs.nju.edu.cn/rinc/SOINN/e-soinn.pdf)
* [Detailed article](http://www.haselab.info/soinn-e.html)

In [197]:
import numpy as np
import matplotlib.pyplot as plt
import random


First, let's determine the abstract class for *neural network* (NN), which implements the base NN structure with voluntary neuron's types.

In [198]:
class ESOINN_Rib:
    def __init__(self, exist=0):
        self.exist = exist
        self.age = 0

In [199]:
class ESOINN_Neuron:
    def __init__(self, feature_vector=()):
        self.feature_vector = np.array(feature_vector)  
        self.accamulate_signals = 0
        self.total_points = 0
        self.density = 0
        self.subclass_id = -1
    
    def update_accamulate_signals(self, n=1):
        self.accamulate_signals += 1

In [207]:
class ESOINN_NN:
    def __init__(self, init_neurons, C1=0.001, C2=1, learning_step=200, max_age=50, 
                 metrics=lambda x,y,axis=1: np.sqrt(np.sum(np.square(np.array(x) - np.array(y)), axis=axis))):
        self.C1 = C1
        self.C2 = C2
        self.learning_step = learning_step
        self.max_age = max_age
        self.signals_amount = 2
        self.metrics = metrics
        
        self.neurons = np.array([
            ESOINN_Neuron(init_neurons[0]),
            ESOINN_Neuron(init_neurons[1])
        ], dtype=ESOINN_Neuron)
        
        self.adjacency_matrix = np.array([
            np.array([
                ESOINN_Rib(),
                ESOINN_Rib()
            ]), 
            np.array([
                ESOINN_Rib(),
                ESOINN_Rib()
            ])
        ], dtype=ESOINN_Rib)
        
        
    
    def fit(self, input_signal):
        self.signals_amount += 1
        
        winners_indexes, distances = self.find_winners(input_signal)
        
        thresholds = [
            self.calc_threshold(input_signal, winners_indexes[0]),
            self.calc_threshold(input_signal, winners_indexes[1])
        ]
        
        if distances[0] > thresholds[0] or distances[1] > thresholds[1]:
            self.create_neuron(input_signal)
            return
        
        self.update_ribs_age(winners_indexes[0], 1)
        
        self.build_connection(winners_indexes)
        
        self.update_density(winners_indexes[0])
        
        self.neurons[winners_indexes[0]].update_accamulate_signals()
        
        self.update_feature_vector(winners_indexes[0])
        
        self.remove_old_ages()
        
        if self.signals_amount % self.learning_step == 0:
            self.update_topology()
            
    def predict(self, input_signal):
        pass
    
    def update(self):
        pass
    
    def find_winners(self, input_signal):
        distances = self.metrics(input_signal, [neuron.feature_vector for neuron in self.neurons])
        
        first_winner = float('inf')
        second_winner = float('inf')
        for dist in distances:
            if dist <= first_winner:
                first_winner, second_winner = dist, first_winner
            elif dist < second_winner:
                second_winner = dist
        
        return [list(distances).index(first_winner), list(distances).index(second_winner)], [first_winner, second_winner]
                        
    
    def calc_threshold(self, input_signal, winner_index):
        target_row = self.adjacency_matrix[winner_index]
        if np.array([rib.exist for rib in target_row]).any():
            return np.max([
                self.metrics(self.neurons[winner_index].feature_vector, self.neurons[rib_index].feature_vector, axis=0) 
                for rib_index, rib in enumerate(target_row) 
                if rib.exist
            ])
        else:
            return self.find_winners(self.neurons[winner_index].feature_vector)[1][1]  # 'cause first winner is always current node
    
    def create_neuron(self, input_signal):
        
        self.neurons = np.append(self.neurons, ESOINN_Neuron(input_signal))
        
#         @fixThisFuckingBullshit
        self.adjacency_matrix = np.hstack([
            self.adjacency_matrix, 
            [[ESOINN_Rib()] for i in range(self.adjacency_matrix.shape[0])]
        ])
        self.adjacency_matrix = np.vstack([
            self.adjacency_matrix, 
            [ESOINN_Rib() for i in range(self.adjacency_matrix.shape[1])]  # use here updated form of matrix
        ])
    
    def update_ribs_age(self, neuron_index, step):
        pass
    
    def build_connection(self, winners_indexes):
        pass
    
    def update_density(self, neuron_index):
        pass
    
    def update_feature_vector(self, neuron_index):
        pass
    
    def remove_old_ages(self):
        pass
    
    def current_state(self):
        pass

In [210]:
input_signal = np.array([2, 19])
nn = ESOINN_NN([[1, 2], [5, 2]])

nn.fit(input_signal)

for i in range(nn.neurons.shape[0]):
    print(nn.neurons[i].feature_vector)


[1 2]
[5 2]
[ 2 19]


In [195]:
a = np.array([
    [1,2],
    [3,4]
])
# a = np.hstack([a, [[5], [7]]])
# a

print(a.shape)
a = np.expand_dims(a,)
a

(2, 2)


array([[1, 2],
       [3, 4]])